This file will be used to contain data processing components

In [2]:
%pip install boto3 pyspark delta-spark python-dotenv

  Obtaining dependency information for botocore<1.35.0,>=1.34.69 from https://files.pythonhosted.org/packages/a9/2e/60f206e1cf4c7d39c0be409b3ebc718f32ac8cb6af89a92fa24a823f9ca7/botocore-1.34.71-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.0 MB 1.3 MB/s eta 0:00:10
   - -------------------------------------- 0.4/12.0 MB 2.8 MB/s eta 0:00:05
   -- ------------------------------------- 0.8/12.0 MB 4.9 MB/s eta 0:00:03
   --- ------------------------------------ 0.9/12.0 MB 4.6 MB/s eta 0:00:03
   ----- ---------------------------------- 1.7/12.0 MB 6.6 MB/s eta 0:00:02
   ------ --------------------------------- 1.9/12.0 MB 6.0 MB/s eta 0:00:02
   --------- ------------------------------ 2.7/12.0 MB 7.9 MB/s eta 0:00:02
   ---------- ----------------------------- 3.3/12.0 MB 8.4 MB/s eta 0:00:02
   ------------ --------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.1 requires botocore<1.34.52,>=1.34.41, but you have botocore 1.34.71 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:

import os
from pyspark.sql import SparkSession
from pyspark import SparkConf

In [4]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')

In [5]:
path_1 = "s3a://data/data-raw/data.json"
path_2 = "s3a://data/data-raw/data2.json"
path_3 = "s3a://data/data-raw/data3.json"

In [6]:
conf = SparkConf()
conf.set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4")
conf.set("spark.hadoop.fs.s3a.access.key", obj_storage_access_key)
conf.set("spark.hadoop.fs.s3a.secret.key", obj_storage_secret_key)
conf.set("spark.hadoop.fs.s3a.endpoint", obj_storage_endpoint)
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider')
conf.set("fs.s3a.requester.pays.enabled", "true")

In [7]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config(conf=conf) \
    .getOrCreate()

In [47]:
from functools import reduce
path_list = [path_1, path_2, path_3]
dataframes = map(lambda p: spark.read.json(p, multiLine=True), path_list)
# IF NULL VALUES NEED TO BE DELETED:
# dataframes = map(lambda p: spark.read.json(p, multiLine=True, if), path_list)
union_df = reduce(lambda df1, df2: df1.union(df2), dataframes)


In [17]:
union_df.show()

+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|             batters|             filling|  id|               name| ppu|             topping| type|
+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|{[{1001, Regular}...|                NULL|0004|              Jelly|0.65|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0005|     Custard-Filled|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|[{6001, None}, {6...|0006|     Cinnamon Twist|0.85|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0007|    Vanilla Frosted|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0008| Strawberry Frosted|0.85|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0009|     Chocolate Cake|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0010|     Blueberry Cake|0.55|[{5001, None}, {5.

In [48]:
import pandas as pd
pandas_df = union_df.toPandas()
pandas_df.to_json(path_or_buf="result.json", orient='records')

In [45]:
# THIS PART IS A TEST TO FORMAT RESULT.JSON
# import json
# with open(file="result.json", mode="r+") as file:
#     data = json.load(file)
#     for rec in data:
        
#         rec["batters"] = {"batter": item for item in rec["batters"]}
#     file.seek(0)
#     json.dump(data, file, indent=4)

In [50]:
def upload_file_to_minio(file_path, minio_bucket, minio_object_name):
    s3c = boto3.resource('s3',
                        endpoint_url=obj_storage_endpoint,
                        aws_access_key_id=obj_storage_access_key,
                        aws_secret_access_key=obj_storage_secret_key,
                        config=boto3.session.Config(signature_version='s3v4'),
                        verify=False
                        )
    s3c.Bucket(minio_bucket).upload_file(file_path, minio_object_name)

In [52]:
upload_file_to_minio("result.json", "data", "data-result/result.json")

In [54]:
result_path = "s3a://data/data-result/result.json"
result_df = spark.read.json(result_path, multiLine=True)
result_df.show()

+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|             batters|             filling|  id|               name| ppu|             topping| type|
+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|[[[1001, Regular]...|                NULL|0004|              Jelly|0.65|[[5001, None], [5...|donut|
|[[[1001, Regular]...|                NULL|0005|     Custard-Filled|0.75|[[5001, None], [5...|donut|
|[[[1001, Regular]...|[[6001, None], [6...|0006|     Cinnamon Twist|0.85|[[5001, None], [5...|donut|
|[[[1001, Regular]...|                NULL|0007|    Vanilla Frosted|0.75|[[5001, None], [5...|donut|
|[[[1001, Regular]...|                NULL|0008| Strawberry Frosted|0.85|[[5001, None], [5...|donut|
|[[[1001, Regular]...|                NULL|0009|     Chocolate Cake|0.75|[[5001, None], [5...|donut|
|[[[1001, Regular]...|                NULL|0010|     Blueberry Cake|0.55|[[5001, None], [5.